In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [0]:
import numpy as np

In [0]:
def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

bandits = [0.2, 0 , -0.2, -5] 
num_bandits = len(bandits)

In [0]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

responsible_weight = tf.slice(weights, action_holder,[1])

loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [0]:
total_episodes = 1000 
total_reward = np.zeros(num_bandits) 
e = 0.1  

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
  
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action])
        
        _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        total_reward[action] += reward
        if i % 50 == 0:
            print( "Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1

Running reward for the 4 bandits: [1. 0. 0. 0.]
Running reward for the 4 bandits: [ 0.  2. -1. 42.]
Running reward for the 4 bandits: [ 0.  1. -2. 90.]
Running reward for the 4 bandits: [  1.  -1.  -2. 137.]
Running reward for the 4 bandits: [  1.  -1.  -3. 186.]
Running reward for the 4 bandits: [ -1.   1.  -3. 232.]
Running reward for the 4 bandits: [ -1.   1.  -3. 282.]
Running reward for the 4 bandits: [ -1.   1.  -2. 331.]
Running reward for the 4 bandits: [ -2.   0.  -2. 375.]
Running reward for the 4 bandits: [ -2.   0.  -2. 425.]
Running reward for the 4 bandits: [ -1.   1.   0. 471.]
Running reward for the 4 bandits: [ -1.   0.   0. 518.]
Running reward for the 4 bandits: [ -3.   0.   1. 565.]
Running reward for the 4 bandits: [ -3.   1.   2. 613.]
Running reward for the 4 bandits: [ -3.   1.   2. 661.]
Running reward for the 4 bandits: [ -4.   3.   5. 705.]
Running reward for the 4 bandits: [ -4.   1.   4. 752.]
Running reward for the 4 bandits: [ -5.   1.   6. 799.]
Running 

In [0]:
print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...and it was right!")
else:
    print ("...and it was wrong!")

The agent thinks bandit 4 is the most promising....
...and it was right!
